Выбираем Трек A: Question Answering (QA)

Оценки качества Вопрос-ответ в дальнейше будет интересно применять при разработке чат-ботов.

Устанавливаем библиотеку для работы с датасетами и метриками

In [67]:
%%sh
pip install datasets evaluate transformers sentence-transformers

Загружаем датасет SberQuAD

In [2]:
from datasets import load_dataset

# Загрузка датасета
dataset = load_dataset("kuznetsoffandrey/sberquad")

# Проверка структуры
print(dataset)
print("\nПример данных:")
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

Пример данных:
{'id': 62310, 'title': 'SberChallenge', 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Сев

Устанавливаем openai

In [28]:
%%sh
  pip install openai

Пишем функцию получения ответов от модели на вопросы по предложеному тексту
и проверяем ее

In [73]:
def getOpenAiAnswerFromContext(prompt: str, model: str) -> str:

    import os
    import openai

    YANDEX_CLOUD_FOLDER = os.getenv("folder_id")
    YANDEX_CLOUD_API_KEY = os.getenv("ya_token")

    client = openai.OpenAI(
        api_key=YANDEX_CLOUD_API_KEY,
        base_url="https://llm.api.cloud.yandex.net/v1"
    )

    # Формируем системный промпт
    system_prompt = (
        "Ты — помощник по вопросам и ответам. Твоя задача — внимательно прочитать предоставленный текст и дать точный ответ на вопрос, используя только информацию из этого текста."
    )

    response = client.chat.completions.create(
        model=f"gpt://{YANDEX_CLOUD_FOLDER}/{model}",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.9,  # делаем меньше температуру чтобы модель не придумывала
        stream=False
    )

    # Извлекаем ответ
    answer = response.choices[0].message.content.strip()


    return answer


# Пример использования
example_context = "Москва — столица России. Она расположена на берегу реки Москвы."
example_question = "Где находится Москва?"

prompt = f"Текст: {example_context}\nВопрос: {example_question}"

print(prompt)

answer = getOpenAiAnswerFromContext(prompt, 'gpt-oss-20b/latest')
print("Ответ:", answer)

Текст: Москва — столица России. Она расположена на берегу реки Москвы.
Вопрос: Где находится Москва?
Ответ: Москва находится на берегу реки Москвы.


Будем использовать следующие модели доступные в yandex.cloud

gpt-oss-20b/latest

gemma-3-27b-it/latest

llama-lite

yandexgpt-lite

Весь списко моделей тут https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models#generation


Запускаем контейнер с mlflow


In [30]:
%%sh
pip install mlflow

In [32]:
%%sh
docker run --rm -d -p 5000:5000 --name=mlflow -v mlflow:/app mlflow:2.0

0ea7c011b906b415981185ef26bb877c456bc0c1f93472795a5254ee9ebe5f3b


In [74]:
import mlflow
from evaluate import load
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer, util
import time

#размер части из датасета
chunk = 30


# Инициализация MLFlow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"QA_Model_Benchmark_0.9_{datetime.now().strftime('%Y%m%d_%H%M%S')}")

# Загрузка метрик
bleu_metric = load("bleu")
squad_metric = load("squad")

# Загрузка модели Sentence-BERT для Semantic Similarity
semantic_model = SentenceTransformer('bert-base-nli-mean-tokens')

# Сохраняем подмножество датасета
subset = dataset['train'].select(range(chunk))
subset_df = subset.to_pandas()
subset_df.to_csv("dataset_subset.csv", index=False)

# Список для хранения результатов
results = []

for model in ['gpt-oss-20b/latest', 'gemma-3-27b-it/latest', 'llama-lite', 'yandexgpt-lite']:
#for model in ['gpt-oss-20b/latest']:

    # Начало нового эксперимента для модели
    with mlflow.start_run(run_name=f"Model_{model}"):
        # Логируем параметры
        mlflow.log_param("model", model)
        mlflow.log_param("dataset_name", "kuznetsoffandrey/sberquad")
        mlflow.log_param("semantic_model", "bert-base-nli-mean-tokens")

        # Логируем подмножество датасета
        mlflow.log_artifact("dataset_subset.csv")

        # Переменные для подсчета средних метрик
        total_bleu = 0
        total_em = 0
        total_f1 = 0
        total_semantic = 0
        total_examples = 0
        total_answer_len = 0
        total_inference_time = 0

        for i in range(chunk):

            # Получаем контекст и вопрос
            context = dataset['train'][i]['context']
            question = dataset['train'][i]['question']
            answer_validate = dataset['train'][i]['answers']['text'][0]

            prompt = f"Текст: {context}\nВопрос: {question}"
            mlflow.log_metric(f"example_{i}_prompt_len", len(prompt))

            start_time = time.time()
            # Вызываем функцию
            answer_model = getOpenAiAnswerFromContext(prompt, model)
            end_time = time.time()
            inference_time = end_time - start_time
            mlflow.log_metric(f"example_{i}_inference_time", inference_time)
            total_inference_time += inference_time


            # Логируем длину ответа модели
            answer_len = len(answer_model.strip())  # Длина ответа (без пробелов)
            mlflow.log_metric(f"example_{i}_answer_len", answer_len)
            total_answer_len += answer_len

            #вычисляем BLEU
            bleu_score = bleu_metric.compute(
            references=[answer_validate] ,
            predictions=[answer_model],
            max_order=1  # BLEU-1 (можно изменить на 2, 3, 4 для более высоких порядков)
            )["bleu"]

            # Логируем BLEU для конкретного примера
            mlflow.log_metric(f"example_{i}_bleu", bleu_score)

            # Вычисляем SQuAD метрики (F1 и EM)
            predictions = [{"id": str(i), "prediction_text": answer_model}]
            references  = [{"id": str(i), "answers": {"text": [answer_validate], "answer_start": [0]}}]

            squad_results = squad_metric.compute(predictions=predictions, references=references)

            em_score = squad_results["exact_match"]
            f1_score = squad_results["f1"]


            # Логируем метрики
            mlflow.log_metric(f"example_{i}_em", em_score)
            mlflow.log_metric(f"example_{i}_f1", f1_score)

            # Вычисляем Semantic Similarity
            if answer_validate.strip() and answer_model.strip():
                embeddings_validate = semantic_model.encode(answer_validate, convert_to_tensor=True)
                embeddings_model = semantic_model.encode(answer_model, convert_to_tensor=True)
                semantic_score = util.cos_sim(embeddings_validate, embeddings_model).item()
            else:
                semantic_score = 0.0  # Если один из ответов пустой

            mlflow.log_metric(f"example_{i}_semantic", semantic_score)


            # Суммируем для подсчета среднего
            total_bleu += bleu_score
            total_em += em_score
            total_f1 += f1_score
            total_semantic += semantic_score
            total_examples += 1

            # Сохраняем результат
            results.append({
                "question": question,
                "answer_validate": answer_validate,
                "answer_model": answer_model,
                "answer_len": answer_len,
                "inference_time": inference_time,
                "semantic_similarity": semantic_score,
                "bleu_score": bleu_score,
                "em_score": em_score,
                "f1_score": f1_score,
            })

        # Логируем средние метрики
        avg_bleu = total_bleu / total_examples
        avg_em = total_em / total_examples
        avg_f1 = total_f1 / total_examples
        avg_semantic = total_semantic / total_examples
        avg_answer_len = total_answer_len / total_examples
        avg_inference_time = total_inference_time / total_examples
        mlflow.log_metric("average_bleu", avg_bleu)
        mlflow.log_metric("average_em", avg_em)
        mlflow.log_metric("average_f1", avg_f1)
        mlflow.log_metric("average_semantic", avg_semantic)
        mlflow.log_metric("average_answer_len", avg_answer_len)
        mlflow.log_metric("average_inference_time", avg_inference_time)


        # Создаем DataFrame из результатов
        df = pd.DataFrame(results)
        df.to_html("model_answers.html", index=False)
        mlflow.log_artifact("model_answers.html")

    print(f"[{i+1}/{chunk}] Успешно обработано")


    mlflow.end_run()

print(results)


2025/09/06 17:41:28 INFO mlflow.tracking.fluent: Experiment with name 'QA_Model_Benchmark_0.9_20250906_174128' does not exist. Creating a new experiment.


🏃 View run Model_gpt-oss-20b/latest at: http://localhost:5000/#/experiments/174819170067133907/runs/d9e0250cd0ba4107ae03cef0bc51b469
🧪 View experiment at: http://localhost:5000/#/experiments/174819170067133907
[30/30] Успешно обработано
🏃 View run Model_gemma-3-27b-it/latest at: http://localhost:5000/#/experiments/174819170067133907/runs/002bd120cc0449c0bc62d223c8f5305a
🧪 View experiment at: http://localhost:5000/#/experiments/174819170067133907
[30/30] Успешно обработано
🏃 View run Model_llama-lite at: http://localhost:5000/#/experiments/174819170067133907/runs/5212e52779a340679f20240b2926cecb
🧪 View experiment at: http://localhost:5000/#/experiments/174819170067133907
[30/30] Успешно обработано
🏃 View run Model_yandexgpt-lite at: http://localhost:5000/#/experiments/174819170067133907/runs/e9e39a703bea4e35a96017f1804a445b
🧪 View experiment at: http://localhost:5000/#/experiments/174819170067133907
[30/30] Успешно обработано
[{'question': 'чем представлены органические остатки?', 'answ

In [ ]:
Запускаем код 3 раза для разной температуры генерации  (0.1, 0.5, 0.9)